In [1]:
import pandas as pd
from math import log
import os

cwd = os.getcwd()

print("Current working directory: {0}".format(cwd))

print ("os.getcwd() returns an object of type {0}".format(type(cwd)))

# copy the filepath
os.chdir ("________")

df = pd.read_csv('Task 3 and 4_Loan_Data.csv')

x = df['default'].to_list()
y = df['fico_score'].to_list()
n = len(x)
print (len(x), len(y))

default = [0 for i in range(851)]
total = [0 for i in range(851)]

for i in range(n):
    y[i] = int(y[i])
    default[y[i]-300] += x[i]
    total[y[i]-300] += 1

for i in range(0, 551):
    default[i] += default[i-1]
    total[i] += total[i-1]

import numpy as np

def log_likelihood(n, k):
    p = k/n
    if (p==0 or p==1):
        return 0
    return k*np.log(p)+ (n-k)*np.log(1-p)

r = 10
dp = [[[-10**18, 0] for i in range(551)] for j in range(r+1)]

for i in range(r+1):
    for j in range(551):
        if (i==0):
            dp[i][j][0] = 0
        else:
            for k in range(j):
                if (total[j]==total[k]):
                    continue
                if (i==1):
                    dp[i][j][0] = log_likelihood(total[j], default[j])
                else:
                    if (dp[i][j][0] < (dp[i-1][k][0] + log_likelihood(total[j]-total[k], default[j] - default[k]))):
                        dp[i][j][0] = log_likelihood(total[j]-total[k], default[j]-default[k]) + dp[i-1][k][0]
                        dp[i][j][1] = k

print (round(dp[r][550][0], 4))

k = 550
l = []
while r >= 0:
    l.append(k+300)
    k = dp[r][k][1]
    r -= 1

print(l)

MSE buckets (ascending FICO):
      i     j  low  high    n    k     p_hat  mean_score            sse
0     0   392  300   415  393  178  0.452926  358.872774  463591.638677
1   393   782  416   526  390  188  0.482051  471.725641  390921.643590
2   783  1187  527   634  405  125  0.308642  580.940741  407996.577778
3  1188  1590  635   742  403   11  0.027295  688.449132  393839.707196
4  1591  1999  743   850  409    1  0.002445  797.000000  412482.000000

Log-likelihood buckets (ascending FICO):
      i     j  low  high    n    k     p_hat  mean_score      loglik
0     0   913  300   562  914  421  0.460699  433.415755 -630.697710
1   914  1088  562   610  175   56  0.322034  586.445714 -109.703815
2  1089  1203  611   638  115   17  0.153846  623.530435  -48.191937
3  1204  1396  638   691  193    7  0.041026  664.528497  -30.146621
4  1397  1999  692   850  603    2  0.004959  771.109453  -13.600811

Rating map (MSE -> rating 1 best):
   rating  low  high     p_hat    n    k  mean